# Data Preparation

Types of missingness: **Missing Completely At Random: MCAR**, **MAR: Missing at random**, **MNAR: Missing not at random**.

### Details on Missingness Types
- **MCAR**: data missing completely at random.
- **MAR**: missing depends on observed features.
- **MNAR**: missing depends on unobserved values.


### Data Quality and Tidiness
- Remove duplicates and inconsistent types.
- Standardize column names.
- Ensure each variable forms a column and each observation a row.


In [1]:
# Load dataset
import pandas as pd
df = pd.read_csv('../data/house_prices.csv')
# View first rows
df.iloc[:3]

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,SalePrice
0,-1.407293,-0.956059,2.317637,0.125549,-1.298104,-0.724861,0.054052,0.810027,-78.194980
1,0.087016,-0.773456,1.898735,0.298798,-1.036144,-2.085792,0.973726,-0.343206,55.700375
2,1.322744,-1.496159,-0.918212,0.456688,-0.169123,0.910418,1.573734,-0.605156,134.355691


### Dataset Overview
The synthetic housing dataset contains eight numerical features representing attributes like size, lot parameters, and neighborhood indices. The target column `SalePrice` is a continuous variable used for regression examples.

In [ ]:
# Visualize target distribution
import matplotlib.pyplot as plt
import seaborn as sns
sns.histplot(df['SalePrice'], kde=True)
plt.title('Distribution of SalePrice')
plt.show()

In [ ]:
# Correlation matrix
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train shape:', X_train.shape)
print('Test shape:', X_test.shape)

### Exploring with pandas and NumPy

In [ ]:
# Basic DataFrame information
print('Shape:', df.shape)
print('Columns:', df.columns.tolist())

In [ ]:
# Filter rows where SalePrice is above the median
df[df['SalePrice'] > df['SalePrice'].median()].head()

In [ ]:
# Selecting specific rows and columns
df.loc[df['Feature1'] < 0, ['Feature1', 'SalePrice']].head()

In [ ]:
# Using iloc for row slicing
df.iloc[:5, :3]

In [ ]:
# NumPy operations on columns
import numpy as np
arr = df[['Feature1', 'Feature2']].values
print('Means:', np.mean(arr, axis=0))
arr[arr[:,0] > 0][:3]

Impute missing values using median vs mean.

### Mean vs Median
Mean: $$\mu = \frac{1}{n} \sum_{i=1}^n x_i$$
Median: value splitting the ordered data in half.
**Median advantages**: robust to outliers.
**Mean advantages**: uses all values but sensitive to outliers.

In [2]:
# Median imputation for missing values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
imp.fit_transform(df)[:3]

array([[-1.40729344e+00, -9.56058552e-01,  2.31763679e+00,
         1.25548587e-01, -1.29810390e+00, -7.24861084e-01,
         5.40516628e-02,  8.10027239e-01, -7.81949799e+01],
       [ 8.70164804e-02, -7.73455844e-01,  1.89873498e+00,
         2.98797780e-01, -1.03614357e+00, -2.08579244e+00,
         9.73726231e-01, -3.43205927e-01,  5.57003754e+01],
       [ 1.32274447e+00, -1.49615881e+00, -9.18212167e-01,
         4.56688011e-01, -1.69122686e-01,  9.10418500e-01,
         1.57373422e+00, -6.05156236e-01,  1.34355691e+02]])

One-hot encoding example:

In [3]:
# One-hot encode categorical variables
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)
enc.fit_transform([[0],[1],[0]])

array([[1., 0.],
       [0., 1.],
       [1., 0.]])

Scaling features:

### Standardization Formula
Each feature x is scaled using:
$$z = {x - \mu}/{\sigma}$$
This results in zero mean and unit variance.

In [4]:
# Standardize numeric features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled = sc.fit_transform(df)
scaled[:3]

array([[-1.34584062, -1.03274708,  2.2987294 ,  0.13847038, -1.29692062,
        -0.69390585,  0.09128125,  0.81322727, -0.43714452],
       [ 0.09943212, -0.84653693,  1.88566917,  0.31005175, -1.040968  ,
        -2.05324327,  1.0209649 , -0.3394965 ,  0.33693157],
       [ 1.29460853, -1.58351741, -0.8919957 ,  0.46642202, -0.19383109,
         0.93945847,  1.62750294, -0.6013311 ,  0.79165383]])

ColumnTransformer pipeline:

### Pipeline Steps
1. Choose numerical and categorical columns.
2. Create preprocessing pipelines for each type.
3. Combine them using `ColumnTransformer`.
4. Fit on training data and transform new inputs.

In [5]:
# Build preprocessing pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
num = df.columns[:-1]
pipe = Pipeline([('imputer', SimpleImputer()), ('scaler', StandardScaler())])
ct = ColumnTransformer([('num', pipe, num)])
ct

,transformers,"[('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


### Usage Example

In [6]:
# Apply transformations
ct.fit_transform(df)[:3]

array([[-1.34584062, -1.03274708,  2.2987294 ,  0.13847038, -1.29692062,
        -0.69390585,  0.09128125,  0.81322727],
       [ 0.09943212, -0.84653693,  1.88566917,  0.31005175, -1.040968  ,
        -2.05324327,  1.0209649 , -0.3394965 ],
       [ 1.29460853, -1.58351741, -0.8919957 ,  0.46642202, -0.19383109,
         0.93945847,  1.62750294, -0.6013311 ]])

### Exercises & Further Reading
1. Try different imputers.
2. Compare `StandardScaler` vs `MinMaxScaler`.
3. [scikit-learn preprocessors](https://scikit-learn.org/stable/modules/preprocessing.html)
4. Use Pandas to check for missing values and fill them with a custom constant.
5. Compute column statistics with NumPy before and after scaling.
6. Plot histograms of a feature before and after imputation using matplotlib.
7. Create seaborn boxplots to visualize the effect of scaling.
8. Plot a correlation heatmap with seaborn after preprocessing.
9. Manually normalize features with NumPy and compare results.
10. Save the cleaned DataFrame using Pandas.
11. Filter houses with `SalePrice` above the mean and compute their average `Feature1`.
12. Use `.loc` to retrieve the `SalePrice` for rows 100 to 110.
13. Select the last 5 rows using `.iloc`.
14. Use NumPy to compute the correlation between `Feature1` and `Feature2`.
15. Plot a histogram of `SalePrice` for the filtered houses.
16. Save the filtered DataFrame to disk with Pandas.

### Hints
- `SimpleImputer` can use `mean` or `median` strategies.
- Replace `StandardScaler` with `MinMaxScaler` to scale between 0 and 1.
- Inspect missing values with `df.isna().sum()`.
- `sns.histplot` and `sns.boxplot` help visualize distributions.
- Save results using `df.to_csv('clean.csv', index=False)`.